# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.debugger import set_trace
import json
from matplotlib import colors
from ast import literal_eval

# Load data

In [2]:
# psynet 11
fname = "sing-scales-v1"
fname = "sing-scales-v2"

PATH = f"data/{fname}/"
nodes = pd.read_csv(PATH + "CustomNode.csv", low_memory=False)
# networks = pd.read_csv(PATH + "CustomNetwork.csv", low_memory=False) # old psynet
trials = pd.read_csv(PATH + "CustomTrial.csv", low_memory=False)

In [3]:
print(nodes)

     vars    id        creation_time  failed  failed_reason  time_of_death  \
0      {}    17  2024-11-21 13:47:55   False            NaN            NaN   
1      {}    18  2024-11-21 13:47:55   False            NaN            NaN   
2      {}    19  2024-11-21 13:47:55   False            NaN            NaN   
3      {}    20  2024-11-21 13:47:55   False            NaN            NaN   
4      {}    21  2024-11-21 13:47:55   False            NaN            NaN   
...   ...   ...                  ...     ...            ...            ...   
2390   {}  3512  2024-11-22 19:05:44   False            NaN            NaN   
2391   {}  3519  2024-11-22 19:09:54   False            NaN            NaN   
2392   {}  3520  2024-11-22 19:10:12   False            NaN            NaN   
2393   {}  3521  2024-11-22 19:10:28   False            NaN            NaN   
2394   {}  3522  2024-11-22 19:10:50   False            NaN            NaN   

             type  network_id  participant_id   trial_maker_id 

In [4]:
# filter nodes
node_data = nodes
node_data = node_data[node_data["failed"] == False]
node_data = node_data[["id", "network_id", "degree", "trial_maker_id", "definition", "seed"]]
node_data = node_data.sort_values(["network_id", "degree"])
node_data["definition"].iloc[1]

def find_end_tt(d, end_tt_list=None):
    if end_tt_list is None:
        end_tt_list = []
    if isinstance(d, dict):
        for key, value in d.items():
            if key == "start_tt":
                end_tt_list.append(value)
            elif isinstance(value, (dict, list)):
                find_end_tt(value, end_tt_list)
    elif isinstance(d, list):
        for item in d:
            find_end_tt(item, end_tt_list)
    return end_tt_list


node_data['start_tt'] = node_data['definition'].apply(lambda x: find_end_tt(json.loads(x)))

print(node_data['start_tt'])


0                                                      []
345     [710.9070294784581, 1728.5487528344672, 2321.9...
451     [1023.3333333333334, 2205.532879818594, 2885.8...
624     [1157.5056689342405, 2367.3469387755104, 2989....
701     [1296.9160997732426, 2530.0680272108843, 3176....
                              ...                        
2188    [1086.8707482993198, 1858.9795918367347, 2783....
2242    [914.2630385487529, 1697.5736961451248, 2483.9...
2288    [1052.81179138322, 1847.8684807256236, 2610.31...
2319    [1086.1678004535147, 1799.7278911564626, 2619....
2349    [1271.315192743764, 2078.185941043084, 2817.02...
Name: start_tt, Length: 2395, dtype: object


# Preprocess data

In [5]:
# filter nodes
node_data = nodes
node_data = node_data[node_data["failed"] == False]
node_data = node_data[["id", "network_id", "degree", "trial_maker_id", "definition", "seed"]]
node_data = node_data.sort_values(["network_id", "degree"])

# process node definition
mel_id = []
# register = []
trial_type = []
target_pitches = []
# reference_pitch = []
target_intervals = []
target_note_durations = []
# target_ISIs = []

for i in range(len(node_data)):
    print()
    # definition = node_data["definition"].to_numpy()[i]
    # definition = literal_eval(definition)
    # definition = [d["content"] for d in definition if d["is_center"]][0]

    definition_str = node_data["definition"].iloc[i]  # Access the string directly
    print(definition_str)
    definition = json.loads(definition_str)  # Parse JSON string into dictionary
    # definition = [d["content"] for d in definition if d["is_center"]][0]
    
    # mel_id.append(definition["mel_id"])
    # register.append(definition["register"])
    trial_type.append(definition["trial_type"])
    target_pitches.append(definition["target_pitches"])
    # reference_pitch.append(definition["reference_pitch"])
    target_intervals.append(definition["target_intervals"])
    target_note_durations.append(definition["target_note_durations"])
    # target_ISIs.append(definition["target_ISIs"])

# node_data["mel_id"] = mel_id
# node_data["register"] = register
node_data["trial_type"] = trial_type
node_data["target_pitches"] = target_pitches
# node_data["reference_pitch"] = reference_pitch
node_data["target_intervals"] = target_intervals
node_data["target_note_durations"] = target_note_durations
# node_data["target_ISIs"] = target_ISIs

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
node_data

,id,network_id,degree,trial_maker_id,definition,seed,trial_type,target_pitches,target_intervals,target_note_durations
0,17,17,0,imitation_chain,"{""register"": ""high"", ""reference_pitch"": 62.131...","{""register"": ""high"", ""reference_pitch"": 62.131...",source_trial,"[55.80546887966199, 56.5906291070845, 65.22516...","[0.7851602274225158, 8.634537612547398, -7.777...","[1.1915190527705082, 0.44400634181100945, 1.10..."
345,452,17,1,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [4...","{""num_target_pitches"": 5, ""target_pitches"": [4...",node_trial,"[43.62377139645649, 47.092592635135404, 51.165...","[3.4688212386789132, 4.07279223477849, -5.1899...","[0.6652154195011337, 0.23653061224489783, 0.79..."
451,607,17,2,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [5...","{""num_target_pitches"": 5, ""target_pitches"": [5...",node_trial,"[56.75154327830167, 61.32748721538579, 63.6473...","[4.575943937084119, 2.3198985753038173, -4.918...","[0.7716553287981859, 0.31668934240362795, 0.82..."
624,860,17,3,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [5...","{""num_target_pitches"": 5, ""target_pitches"": [5...",node_trial,"[56.53617932106461, 60.71894066152018, 62.2838...","[4.182761340455571, 1.5648774297358585, 0.5641...","[0.7872789115646258, 0.30489795918367324, 0.84..."
701,980,17,4,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [4...","{""num_target_pitches"": 5, ""target_pitches"": [4...",node_trial,"[46.46679000325925, 50.10185168191718, 49.0070...","[3.6350616786579337, -1.0948037527414414, 2.31...","[0.6677097505668935, 0.266235827664399, 0.7152..."
...,...,...,...,...,...,...,...,...,...,...
2188,3202,216,7,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [6...","{""num_target_pitches"": 5, ""target_pitches"": [6...",node_trial,"[61.35612242641697, 64.6783813287385, 56.24332...","[3.322258902321529, -8.435056240779616, 1.3968...","[0.5817913832199547, 0.6288662131519275, 0.695..."
2242,3280,216,8,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [6...","{""num_target_pitches"": 5, ""target_pitches"": [6...",node_trial,"[61.58343162257289, 64.74804556006615, 55.0558...","[3.1646139374932574, -9.69224050478482, 3.4544...","[0.6390929705215418, 0.5942630385487527, 0.579..."
2288,3336,216,9,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [6...","{""num_target_pitches"": 5, ""target_pitches"": [6...",node_trial,"[61.09442492379475, 65.40211953948459, 55.5831...","[4.30769461568984, -9.818990236023538, 2.72453...","[0.5582766439909297, 0.5134240362811793, 0.602..."
2319,3385,216,10,imitation_chain,"{""num_target_pitches"": 5, ""target_pitches"": [5...","{""num_target_pitches"": 5, ""target_pitches"": [5...",node_trial,"[59.796899764745206, 65.21656231732484, 57.334...","[5.419662552579638, -7.881870092949313, 2.1214...","[0.4775283446712019, 0.5716099773242631, 0.465..."


# Export node data to csv

In [7]:
node_data.to_csv(f"{fname}_node_data.csv")

# Trial level data

In [8]:
# filter trials
trial_data = trials
trial_data = trial_data[trial_data["failed"] == False]

degrees = []

for i in range(len(trial_data)):
    current_node_id = trial_data.iloc[i]["origin_id"]
    current_nodes = node_data[node_data["id"] == current_node_id]
    current_degree = current_nodes["degree"].to_numpy()[0]
    # answer = d['listen_and_rate_trial_seed']['valence']
    # current_vertex_id = current_nodes["vertex_id"].to_numpy()[0]
    # current_dependent_vertex_ids = current_nodes["dependent_vertex_ids"].to_numpy()[0]
    degrees.append(current_degree)
    # vertex_ids.append(current_vertex_id)
    # dependent_vertex_ids.append(current_dependent_vertex_ids)

trial_data["degree"] = degrees

trial_data = trial_data[["id", "network_id", "degree", "trial_maker_id", "origin_id", "participant_id", "analysis"]]
trial_data = trial_data.sort_values(["network_id", "degree"])

/var/folders/v7/s8wj18ld78gdbf0vgx1ptf7r0000gr/T/ipykernel_31334/4239617211.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_data["degree"] = degrees


In [9]:
trial_data

,id,network_id,degree,trial_maker_id,origin_id,participant_id,analysis
209,604,17,0,imitation_chain,17,42,"{""failed"": false, ""reason"": ""All good"", ""regis..."
341,926,17,1,imitation_chain,452,61,"{""failed"": false, ""reason"": ""All good"", ""regis..."
564,1412,17,2,imitation_chain,607,84,"{""failed"": false, ""reason"": ""All good"", ""regis..."
663,1703,17,3,imitation_chain,860,104,"{""failed"": false, ""reason"": ""All good"", ""regis..."
1494,4321,17,4,imitation_chain,980,301,"{""failed"": false, ""reason"": ""All good"", ""regis..."
...,...,...,...,...,...,...,...
2555,7109,216,6,imitation_chain,3176,473,"{""failed"": false, ""reason"": ""All good"", ""regis..."
2616,7292,216,7,imitation_chain,3202,479,"{""failed"": false, ""reason"": ""All good"", ""regis..."
2666,7365,216,8,imitation_chain,3280,490,"{""failed"": false, ""reason"": ""All good"", ""regis..."
2704,7502,216,9,imitation_chain,3336,499,"{""failed"": false, ""reason"": ""All good"", ""regis..."


# Export trial data to csv

In [10]:
trial_data.to_csv(f"{fname}_trial_data.csv")

# Experiment Cost and Demographics

In [11]:
participants = pd.read_csv(PATH + "participant.csv", low_memory=False)

In [12]:
valid_participants = participants[participants["complete"] == True]
valid_participants = valid_participants[valid_participants["failed"] == False]
experiment_summary = {
    "N_participants": valid_participants.shape[0],
    "cost": participants["base_pay"].sum() + participants["bonus"].sum()
}

experiment_summary

{'N_participants': 142, 'cost': 959.7}